In [2]:
import numpy as np
import pandas as pd
import sys
import os
from codebase.file_utils import save_obj, load_obj
from codebase.post_process import samples_to_df
import altair as alt
alt.data_transformers.disable_max_rows()

%load_ext autoreload
%autoreload 2

In [9]:
ppp = True
# log_dir = "./log/cont_dat_simulation/PPP/sim2/run2/20200527_092145___s2m0/"
log_dir = "./log/20210221_152009_vbtest_s1m1//"
data = load_obj('data', log_dir)


if ppp: 
    ps = load_obj('ps', log_dir)
    ps.keys()
else: 
    print("\n\nChecking data integrity...\n\n")
    complete_data = load_obj("complete_data", log_dir)

    ps = dict()
    ps[0] = load_obj('ps_0', log_dir)
    ps[1] = load_obj('ps_1', log_dir)
    ps[2] = load_obj('ps_2', log_dir)



In [10]:
data

{'random_seed': 0,
 'N': 1000,
 'K': 2,
 'J': 6,
 'alpha': array([0., 0., 0., 0., 0., 0.]),
 'beta': array([[1. , 0. ],
        [0.8, 0. ],
        [0.8, 0. ],
        [0. , 1. ],
        [0. , 0.8],
        [0. , 0.8]]),
 'sigma_z': array([0.80622577, 0.80622577]),
 'Phi_corr': array([[1. , 0.2],
        [0.2, 1. ]]),
 'Phi_cov': array([[0.65, 0.13],
        [0.13, 0.65]]),
 'Marg_cov': array([[1.65  , 0.52  , 0.52  , 0.13  , 0.104 , 0.104 ],
        [0.52  , 1.416 , 0.416 , 0.304 , 0.2832, 0.0832],
        [0.52  , 0.416 , 1.416 , 0.304 , 0.2832, 0.0832],
        [0.13  , 0.304 , 0.304 , 1.65  , 0.52  , 0.72  ],
        [0.104 , 0.2832, 0.2832, 0.52  , 1.416 , 0.616 ],
        [0.104 , 0.0832, 0.0832, 0.72  , 0.616 , 1.416 ]]),
 'Theta': array([[1. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 1. , 0. , 0.2, 0.2, 0. ],
        [0. , 0. , 1. , 0.2, 0.2, 0. ],
        [0. , 0.2, 0.2, 1. , 0. , 0.2],
        [0. , 0.2, 0.2, 0. , 1. , 0.2],
        [0. , 0. , 0. , 0.2, 0.2, 1. ]]),
 'sigma':

In [11]:
ps['alpha'][1,0]

0.010469613422536939

In [12]:
print(open(log_dir+'model.txt').read())

data {
  int<lower=1> N;
  int<lower=1> K;
  int<lower=1> J;
  matrix[N,J] yy;
  vector[J] sigma_prior;
}

transformed data{
  vector[J] zeros = rep_vector(0, J);
  cov_matrix[K] I_K = diag_matrix(rep_vector(1, K));
  real<lower=0> c0 = 2.5;
}

parameters {
  vector<lower=0>[J] sigma_square;
  vector[J] alpha;
  matrix[2,K] beta_free; // 2 free eleements per factor
  cov_matrix [K] Phi_cov;
}

transformed parameters{
  cov_matrix[J] Theta;
  matrix[J,K] beta;
  cov_matrix[J] Marg_cov;
  
  Theta = diag_matrix(sigma_square);

  for(j in 1:J) {
    for (k in 1:K) beta[j,k] = 0;
    }

  // set ones
  for (k in 1:K) beta[1+3*(k-1), k] = 1;

  // set the free elements
  for (k in 1:K) beta[2+3*(k-1) : 3+3*(k-1), k] = beta_free[1:2,k];

  Marg_cov = beta * Phi_cov * beta'+ Theta;

}

model {
  to_vector(beta_free) ~ normal(0, 1);
  to_vector(alpha) ~ normal(0, 10);
  for(j in 1:J) sigma_square[j] ~ inv_gamma(c0, (c0-1)/sigma_prior[j]);
  Phi_cov ~ inv_wishart(J+4, I_K);
  for (n in 1:N){
  

In [16]:

def form_df(samples, rows):
    dfs = []
    for r in range(rows):
        if rows>1:
            df = pd.DataFrame(samples[:,r])
        else:
            df = pd.DataFrame(samples)
        df.insert(0, 'idx', np.arange(df.shape[0]))
        df = df.melt(id_vars ='idx', var_name = 'col')
        df.insert(1, 'row' , r)
        dfs.append(df)
    return pd.concat(dfs).reset_index(drop=True)


def get_post_df(samples):
    if samples.ndim > 2:
        rows = samples.shape[1]
        df = form_df(samples, rows)
    else:
        rows = 1
        df = form_df(samples, 1)
    return df

In [15]:
ps['alpha'].shape

(1000, 6)

In [18]:
# samples_to_df(ps[0], 'alpha').head()
get_post_df(ps['alpha']).head()

,idx,row,col,value
0,0,0,0,-0.069072
1,1,0,0,0.010470
2,2,0,0,-0.094034
3,3,0,0,-0.058337
4,4,0,0,-0.032109


In [22]:
df

,idx,row,col,value
0,0,0,0,1.000000
1,1,0,0,1.000000
2,2,0,0,1.000000
3,3,0,0,1.000000
4,4,0,0,1.000000
...,...,...,...,...
11995,995,5,1,1.120097
11996,996,5,1,1.033092
11997,997,5,1,1.185137
11998,998,5,1,1.015440


In [24]:
df = get_post_df(ps['beta'])

c1 = alt.Chart(df).mark_line(opacity=0.8).encode(
    alt.X('idx:O'),
    alt.Y('value'),
    row = 'row:O',
    column= 'col:O'
).properties(width = 200, height=100, title='Beta') 

c1.interactive()

alt.Chart(...)

In [25]:
np.round(np.mean(np.mean(ps['beta'],0),0),2)

array([0.49, 0.48])

In [29]:
c1 = alt.Chart(get_post_df(ps['alpha'])).mark_line(opacity=0.8).encode(
    alt.X('idx:O'),
    alt.Y('value'),
    row = 'row:O',
    column= 'col:O'
).properties(width = 200, height=100) 

c1.interactive()

alt.Chart(...)